### Readme

* The pre-trained model file is available here - https://drive.google.com/file/d/1zQ0WQ8IugTYGKh5rriCxj_FLusXFuZI4/view?usp=sharing
* for both the below mentioned evaluation/testing methods, you need to download the file, keep the **file as it is with the same name** in the `./model/` directory.

* Both the inference methods are tested on linux machines on CPU with 8GB RAM

**Replicating the inference results on custom data**
1. The easy way to replicate the prediction setup is using docker. See [Custom prediction - method 1](#custom1)
2. If you want to check the prediction without using docker, you may need to follow the instructions given in the [Custom prediction - method 2](#custom2)


*Any queries or bugs? Reach out to renjithbaby23@gmail.com*

<a id='custom1'></a>
## Custom prediction - method 1

**Prediction using the docker**

*If you are not familiad with docker, just follow the instructions. But first install docker from - https://docs.docker.com/engine/install/*

1. You should have docker installed for this to work
2. Clone the repo - `git clone https://github.com/renjithbaby23/sentiment-classification.git`
3. cd to the repo - `cd sentiment-classification`
4. Enusre that you have downloaded the [model file](https://drive.google.com/file/d/1zQ0WQ8IugTYGKh5rriCxj_FLusXFuZI4/view?usp=sharing) and is available at `./model/model.tar.gz`
4. Build the docker image - `docker build ./ -t renjith/sentiment:v1`
5. Run the docker container - `docker run -d --name sentiment-serving -p 80:80 renjith/sentiment:v1`
6. From the same machine where you ran the docker, go to the url -`http://0.0.0.0:80/sentiment-detection/health` to check if the service is up and running.

*If ip 0.0.0.0 is not working on local machine, you can use `http://localhost:80/sentiment-detection/health` instead*

*If you are getting some error related to port already in use, try googling for how to stop the processes running on the same port and you may need to kill them*


Run the below cells to test the model via locally available apis with custom data

In [1]:
# ! pip install requests==2.24.0
import requests
import json

**Modify the string below if you want to test on your own data**

In [2]:
sample_text = 'how can I login to app store?'

In [2]:
# the endpoint that will be created when the docker is running
local_api = "http://0.0.0.0:80/sentiment-detection/predict"
# in case if the above is not working, you can use the below local api
# local_api = "http://localhost:80/sentiment-detection/predict"

# preparing the payload
data = {'content': sample_text}

# get the response
response = requests.request('POST', local_api, json=data)

# parsing the response
prediction = response.json()

print("The predicted sentiment is: '{}' with confidence: {}"\
      .format(prediction['data']['decision'], \
              prediction['data']['prediction confidence']))

The predicted sentiment is: 'neutral' with confidence: 0.89651954


**Cleaning up the docker image and running containers**

1. stop the running container -  `docker stop sentiment-serving`
2. remove the container - `docker rm sentiment-serving`
3. remove the docker image - `docker image rm renjith/sentiment:v1`

<a id='custom2'></a>
## Custom prediction - method 2

**If you want to do the testing manually on custom data, follow the steps**

1. Create a conda environment with python 3.8 `sentiment`
2. Activate the environment `sentiment`
3. Install the requirements `pip install -r requirements.txt`
5. Untar the [model file](https://drive.google.com/file/d/1zQ0WQ8IugTYGKh5rriCxj_FLusXFuZI4/view?usp=sharing) using `tar xzf model.tar.gz`
6. keep the extracted folder **without renaming** - here `./model/apple5_model/`
7. If the TF saved model is kept in a different folder, update the below variable `saved_model_path`
8. Modify the list `test_data` with your custom text to see the predictions
9. Run the below cells of this notebook
10. You can also run the evaluation.py once you are done with the steps till 7

In [6]:
import numpy as np
import tensorflow as tf
import tensorflow_text

In [7]:
saved_model_path = './model/apple5_model/'
class_names = {0: "neutral", 1: "positive", 2:"negative"}

In [8]:
reloaded_model = tf.saved_model.load(saved_model_path)

**You can modify the `test_data` to test the model with your custom text contents**

In [9]:
test_data = [
    'this is such an amazing movie!',
    'The movie was great!',
    'The movie was meh.',
    'The movie was okish.',
    'The movie was terrible...',
    'how can I login to app store?'
]

In [10]:
def print_my_examples(inputs, results):
  result_for_printing = \
    [f'input: {inputs[i]:<30} : score: {results[i]}  : class: {class_names[np.argmax(results[i])]}'
                         for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()

reloaded_results = reloaded_model(tf.constant(test_data))

print('Predictions:')
print_my_examples(test_data, reloaded_results)

Predictions:
input: this is such an amazing movie! : score: [0.00374165 0.99136215 0.00489621]  : class: positive
input: The movie was great!           : score: [0.00444103 0.9867165  0.00884238]  : class: positive
input: The movie was meh.             : score: [0.22289388 0.6971025  0.08000361]  : class: positive
input: The movie was okish.           : score: [0.07919218 0.868262   0.05254583]  : class: positive
input: The movie was terrible...      : score: [0.00729853 0.01884586 0.9738556 ]  : class: negative
input: how can I login to app store?  : score: [0.89651954 0.02201089 0.08146948]  : class: neutral

